##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting with Video

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook provides a quick example of how to prompt Gemini 1.5 Pro using a video file. In this case, you'll use a short clip of [Sherlock Jr.](https://en.wikipedia.org/wiki/Sherlock_Jr.)

In [ ]:
!pip install -U google-generative-ai

In [ ]:
import google.generativeai as genai

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Upload the file

The Gemini API currently does not support video files directly. Instead, you can provide a series of timestamps and image files.

We will extract 1 frame a second from a 10 minute clip of the film [Sherlock Jr.](https://en.wikipedia.org/wiki/Sherlock_Jr.).

Note: You can also [upload your own files](https://github.com/google-gemini/cookbook/tree/main/examples/Upload_files.ipynb) to use.

In [34]:
!curl -o video.mp4 "https://storage.googleapis.com/generativeai-downloads/data/SherlockJr._10min.mp4"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  594M  100  594M    0     0  88.1M      0  0:00:06  0:00:06 --:--:-- 79.5M


In [35]:
f = genai.upload_file("video.mp4")

In [36]:
f.name

'files/hgxvpmisxbn9'

In [37]:
import time

while f.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    f = genai.get_file(f.name)

....

## Generate Content

After the file has been uploaded, you can make `GenerateContent` requests that reference the File API URI.

`GenerateContent` expects a list of inputs. It can operate in two modes either:

1. If the input appears to be a list of `Part` object, it will pack them into a single `Content` object (representing one conversation turn).
2. If the input appears to be a list of `Content` objects each represents one turn of the conversation.

For now, video file references can't be mixed into a list of parts. They need to be contained in their own `Content` object, see below. `generate_content([file, 'describe this file'])` would be valid for any other media type.

In [38]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

response = model.generate_content(
    [f, "Describe this video."],
    request_options={"timeout": 600})
print(response.text)

The video is a black and white silent movie featuring a detective who is also a film projector operator.

The movie begins with a detective reading a note, after which he chases a suspect. He pursues him onto a moving train, then ultimately finds himself stranded on a water tower as the train pulls away. He falls off the water tower and lands directly beneath the water spout. He then gives up on the case and returns to his job as a film projector operator. 

As the projectionist, he falls asleep at the controls, interrupting the movie on-screen. This creates an interesting effect that makes it look like the characters in the film are interacting with the projectionist. He even enters the world of the film on screen and ends up in a brawl with one of the actors. The fight is broken up when the lead actor of the movie calls for a detective, "Sherlock Jr." The video ends with the projectionist being called into work. 



## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
f.delete()

## Learning more

The File API lets you upload a variety of multimodal MIME types, including images and audio formats. The File API handles inputs that can be used to generate content with [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent).

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API.

* Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) with the quickstart.

* Learn more about prompting with [media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length.